In [6]:
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('omw-1.4')
!unzip C:/Users/TharushaLekamge/AppData/Roaming/nltk_data/corpora/wordnet.zip -d C:/Users/TharushaLekamge/AppData/Roaming/nltk_data/corpora/

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tharu\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tharu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tharu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tharu\AppData\Roaming\nltk_data...
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import ParameterGrid
from sklearn.svm import SVC
import sklearn.feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('omw-1.4')
# ##!unzip C:/Users/TharushaLekamge/AppData/Roaming/nltk_data/corpora/wordnet.zip -d C:/Users/TharushaLekamge/AppData/Roaming/nltk_data/corpora/

from bs4 import BeautifulSoup
import re
import pickle

# Importing Shap for shapley values
import shap

loaded_vocab = pickle.load(open('../modelExports/vectorizer_imdb.pkl', 'rb'))

stop_words = set(stopwords.words('english'))
tokenizer = nltk.tokenize.toktok.ToktokTokenizer()
lemmatizer = WordNetLemmatizer() 
loaded_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(min_df=2, vocabulary=loaded_vocab)
label_binarizer = sklearn.preprocessing.LabelBinarizer()
loaded_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(min_df=2, vocabulary=loaded_vocab)

In [2]:
## Support functions
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stop_words]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def lemmatize_text(text):
    words=word_tokenize(text)
    edited_text = ''
    for word in words:
        lemma_word=lemmatizer.lemmatize(word)
        extra=" "+str(lemma_word)
        edited_text+=extra
    return edited_text

## GET INPUT DATA
data = pd.read_csv('../modelExports/IMDB-Dataset.csv')
data = data.sample(10000)

feature_names = loaded_vectorizer.get_feature_names_out()

## Preprocess text

data.review = data.review.str.lower()
data.review = data.review.apply(strip_html)
data.review = data.review.apply(remove_special_characters)
data.review = data.review.apply(remove_stopwords)
data.review = data.review.apply(lemmatize_text)

## Split Data
x_imdb = data['review']
y_imdb = data['sentiment']

x_train_i, x_test_i, y_train_i, y_test_i = train_test_split(x_imdb,y_imdb,test_size=0.2)
x_test, x_val, y_test_i, y_val_i = train_test_split(x_test_i,y_test_i,test_size=0.5)

x_train_imdb = loaded_vectorizer.fit_transform(x_train_i)
x_test_imdb = loaded_vectorizer.transform(x_test)
x_val_imdb = loaded_vectorizer.transform(x_val)

# Binarize y - Positive is 1
y_train_imdb = label_binarizer.fit_transform(y_train_i)
y_test_imdb = label_binarizer.fit_transform(y_test_i)
y_val_imdb = label_binarizer.fit_transform(y_val_i)



The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.


In [4]:
loaded_plain_model_rf = pickle.load(open('../modelExports\grid_imdb_rf.pickle', "rb"))

Trying to unpickle estimator DecisionTreeClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator RandomForestClassifier from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
Trying to unpickle estimator RandomizedSearchCV from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations


In [5]:
#Find the estimators in best model in the randomisedsearchCV loaded_plain_model_rf
estimators = loaded_plain_model_rf.best_estimator_.estimators_
len(estimators)

100

In [6]:
from sklearn import tree

In [7]:
#plt.figure(figsize=(20,20))
#tree.plot_tree(estimators[0], feature_names=feature_names, filled=True, fontsize=10)

# Show the tree in text format
text_representation = tree.export_text(estimators[2])
print(text_representation)

|--- feature_5557 <= 0.01
|   |--- feature_17369 <= 0.05
|   |   |--- feature_26441 <= 0.06
|   |   |   |--- feature_19584 <= 0.05
|   |   |   |   |--- feature_14168 <= 0.03
|   |   |   |   |   |--- feature_15082 <= 0.03
|   |   |   |   |   |   |--- feature_6602 <= 0.05
|   |   |   |   |   |   |   |--- feature_20889 <= 0.03
|   |   |   |   |   |   |   |   |--- feature_1952 <= 0.07
|   |   |   |   |   |   |   |   |   |--- feature_1122 <= 0.01
|   |   |   |   |   |   |   |   |   |   |--- feature_26378 <= 0.01
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 90
|   |   |   |   |   |   |   |   |   |   |--- feature_26378 >  0.01
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 22
|   |   |   |   |   |   |   |   |   |--- feature_1122 >  0.01
|   |   |   |   |   |   |   |   |   |   |--- feature_22873 <= 0.03
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 17
|   |   |   |   |   |   |   |   |   |   |--- feature_2287

In [16]:
# print tree node count of each estimator
# for i in range(len(estimators)):
#     print(estimators[i].tree_.node_count)

# find the features used in each estimator
estimatorsFeatures = []
for i in range(len(estimators)):
    for item in estimators[i].tree_.feature:
        estimatorsFeatures.append(item)

# convert to set to remove duplicates
estimatorsFeatures = set(estimatorsFeatures)


# Find the features where a class change occurs
estimatorsFeatures1 = []
for i in range(len(estimators)):
    for item in estimators[i].tree_.feature:
        if item != -2:
            estimatorsFeatures1.append(item)

set(estimatorsFeatures1) - set(estimatorsFeatures)

set()

In [21]:
# Get the features used in each estimator with its highest class change and label
estimatorsFeatures2 = []
for i in range(len(estimators)):
    for item in estimators[i].tree_.feature:
        if item != -2:
            estimatorsFeatures2.append((item, estimators[i].tree_.value[estimators[i].tree_.feature == item], estimators[i].tree_.value[estimators[i].tree_.feature != item]))

In [22]:
# Get the features used in each estimator with its highest class change and label
estimatorsFeatures3 = []
for i in range(len(estimators)):
    for item in estimators[i].tree_.feature:
        if item != -2:
            estimatorsFeatures3.append((feature_names[item], estimators[i].tree_.value[estimators[i].tree_.feature == item], estimators[i].tree_.value[estimators[i].tree_.feature != item]))

In [42]:
# Get the featuers used in the node just before leaf nodes
estimatorsFeatures4 = []
for i in range(len(estimators)):
    for item in estimators[i].tree_.feature:
        if item == -2:
            if estimators[i].tree_.children_left[item] == -1 and estimators[i].tree_.children_right[item] == -1:
                estimatorsFeatures4.append((feature_names[item], estimators[i].tree_.value[estimators[i].tree_.feature == item], estimators[i].tree_.value[estimators[i].tree_.feature != item]))



TypeError: unhashable type: 'numpy.ndarray'

In [40]:
featureNamesInSet = []
classChangesInSet = []
for i in range(len(estimatorsFeatures3)):
    featureNamesInSet.append(estimatorsFeatures3[i][0])
    classChangesInSet.append(list(estimatorsFeatures3[i][1:]))

featureNamesInSet = list(set(featureNamesInSet))
print(len(featureNamesInSet))

3148


In [38]:
classChangesInSet

[[array([[[4016., 3984.]]]),
  array([[[3.956e+03, 3.976e+03]],
  
         [[3.752e+03, 3.862e+03]],
  
         [[3.737e+03, 3.769e+03]],
  
         ...,
  
         [[3.600e+01, 7.000e+00]],
  
         [[1.800e+01, 1.000e+00]],
  
         [[6.000e+00, 0.000e+00]]])],
 [array([[[3956., 3976.]]]),
  array([[[4.016e+03, 3.984e+03]],
  
         [[3.752e+03, 3.862e+03]],
  
         [[3.737e+03, 3.769e+03]],
  
         ...,
  
         [[3.600e+01, 7.000e+00]],
  
         [[1.800e+01, 1.000e+00]],
  
         [[6.000e+00, 0.000e+00]]])],
 [array([[[3752., 3862.]]]),
  array([[[4.016e+03, 3.984e+03]],
  
         [[3.956e+03, 3.976e+03]],
  
         [[3.737e+03, 3.769e+03]],
  
         ...,
  
         [[3.600e+01, 7.000e+00]],
  
         [[1.800e+01, 1.000e+00]],
  
         [[6.000e+00, 0.000e+00]]])],
 [array([[[3737., 3769.]]]),
  array([[[4.016e+03, 3.984e+03]],
  
         [[3.956e+03, 3.976e+03]],
  
         [[3.752e+03, 3.862e+03]],
  
         ...,
  
         [[3.600e+